In [1]:
import pandas as pd
import openai
import os
import dotenv
from libraries.db import DatabaseProxy, Question
import re
from tqdm import tqdm

In [2]:
with open("resources/question_augment_prompt.txt", "r") as f:
    prompt = f.read()

In [3]:
dotenv.load_dotenv(".envrc")
api_key = os.environ["OPENAI_API_KEY"]
openai.api_key = api_key

In [4]:
db = DatabaseProxy()

In [5]:
questions = db.get_questions()
questions[:3]

[Question(answer_id=1, created_datetime=datetime.datetime(2023, 11, 8, 5, 16, 40), id=1, text='How many candidates do you have in your database?'),
 Question(answer_id=2, created_datetime=datetime.datetime(2023, 11, 8, 5, 16, 40), id=2, text='How often do you review and refresh your database of candidates?'),
 Question(answer_id=3, created_datetime=datetime.datetime(2023, 11, 8, 5, 16, 40), id=3, text='If you are a member of the American Staffing Association, what certifications do you hold?')]

In [6]:
question_pattern = re.compile(r"\d\. (.*)\n?")

for question in tqdm(questions):
    answer = db.get_answer_by_id(question.answer_id)
    content = prompt.format(question=question.text, answer=answer.text)
    conversation = [{"role": "user", "content": content}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation,
        max_tokens=1000,
    )
    text = response["choices"][0]["message"]["content"]
    choices = question_pattern.findall(text)
    choices = [choice.strip().strip('"') for choice in choices]
    for choice in choices:
        new_question = Question(text=choice, answer_id=answer.id)
        db.insert_question(new_question)

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [01:50<00:00,  2.21s/it]


In [7]:
print("\n".join(choices))

Can you provide any instances of claims you've had related to DFEH and EEOC?
Have you encountered any cases involving DFEH and EEOC claims?
What are some examples of DFEH and EEOC issues you've dealt with?
Could you share any experiences you've had with DFEH and EEOC complaints?
In what ways have DFEH and EEOC been involved in your organizational matters?


In [8]:
print(choices)

["Can you provide any instances of claims you've had related to DFEH and EEOC?", 'Have you encountered any cases involving DFEH and EEOC claims?', "What are some examples of DFEH and EEOC issues you've dealt with?", "Could you share any experiences you've had with DFEH and EEOC complaints?", 'In what ways have DFEH and EEOC been involved in your organizational matters?']
